In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import datetime
from tqdm import tqdm
from PIL import Image

import torch
import torch.nn as nn

from torch import optim
from torch.utils.data import Dataset, DataLoader

import sys 
sys.path.append("..") 
from glob import glob
from model.dataset import *

In [2]:
spinal_test_dir = "./DiffusionSpinalMRISynthesis/Data_MRI/test_spinal_MRI"
# test_onepatient_dataset = MRI_patient_Dataset_fortestpatient(dir_path=spinal_test_dir, patient_id=patient_id)

output_path = "./DiffusionSpinalMRISynthesis/results_output_pictures/Ground_True_imgs_test_spinal"
os.makedirs(output_path, exist_ok=True)

def norm(imgdata):
    layers, _, _ = imgdata.shape[0], imgdata.shape[1], imgdata.shape[2]
    max_value = np.max(imgdata.reshape(layers,-1),axis=-1).reshape(layers,1,1)
    min_value = np.min(imgdata.reshape(layers,-1),axis=-1).reshape(layers,1,1)
    imgdata = (imgdata-min_value)/(max_value-min_value+1e-10)
    imgdata = (255 * imgdata).astype(np.uint8) #转成int8
    return imgdata

def Ground_Trueimg_onepatient(patient_id, output_path=output_path):
    spinal_test_dir = "./DiffusionSpinalMRISynthesis//Data_MRI/test_spinal_MRI"
    fnames = glob(os.path.join(spinal_test_dir, '*'+patient_id+'*'))
    test_onepatient_dataset = MRI_patient_Dataset_fortestpatient(dir_path=spinal_test_dir, patient_id=patient_id)

    target_img = torch.concat([test_onepatient_dataset[i][3:,:,:].unsqueeze(dim=0) for i in range(len(test_onepatient_dataset))],dim=0)

    
    target_img = norm(target_img.detach().cpu().squeeze(dim=1).numpy())
    output_img = target_img
    patient_save_path = os.path.join(output_path, patient_id)
    os.makedirs(patient_save_path, exist_ok=True)
    for i in range(target_img.shape[0]):
        img = Image.fromarray(output_img[i,:,:])
        i_savepath = os.path.join(patient_save_path, patient_id+'_T1CE_groundTrue_'+fnames[i].split('_')[-1])
        img.save(i_savepath)

In [3]:
patient_id_list = list(set([path.split('_')[0]+'_'+path.split('_')[1]+'_'+path.split('_')[2] for path in os.listdir(spinal_test_dir)]))
for patient_id in tqdm(patient_id_list):
    Ground_Trueimg_onepatient(patient_id = patient_id)

100%|██████████| 203/203 [01:26<00:00,  2.35it/s]
